In [404]:
from langchain_community.chat_models import ChatOllama
from langchain.embeddings import OllamaEmbeddings

In [405]:
llm = ChatOllama(model = "llama3")
embedding = OllamaEmbeddings(model="nomic-embed-text")

# Document Loading

In [406]:
from langchain.document_loaders import CSVLoader
file_path = "parking_spots.csv"

In [407]:
import pandas as pd
# Load the CSV file using pandas for data inspection
df = pd.read_csv(file_path)

# Sort data by floor and parking number
df_sorted = df.sort_values(by=['Floor', 'Parking Number'])

# Check the first few rows to confirm sorting
print(df_sorted.head())

# Count empty parking spots on the 1st floor
first_floor_empty = df_sorted[(df_sorted['Floor'] == 1) & (df_sorted['Parking Status'] == 'Empty')]
total_empty_first_floor = len(first_floor_empty)
print(f"Total empty parking spots on the 1st floor: {total_empty_first_floor}")



    Floor  Parking Number Parking Status  Parking Type      Metadata
1       1               2         Filled  Special Need  Near to Door
42      1              43          Empty           VIP   Left Corner
43      1              44          Empty  Special Need  Near to Door
45      1              46          Empty  Special Need  Near to Door
46      1              47         Filled           VIP  Near to Door
Total empty parking spots on the 1st floor: 89


In [408]:
# Save the sorted data back to a CSV file
sorted_file_path = "sorted_parking_spots.csv"
df_sorted.to_csv(sorted_file_path, index=False)

In [ ]:
from langchain_core.tools import tool
from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

@tool
def empty_parking_byfloor(floor: str) -> int:
    """Fetch Empty Parking by Floor number"""
    try:
        _floor = int(floor)
    except TypeError as e:
        _floor = str(e)

    first_floor_empty = df_sorted[(df_sorted['Floor'] == _floor) & (df_sorted['Parking Status'] == 'Empty')]
    total_empty_first_floor = len(first_floor_empty)
    print(total_empty_first_floor)
    return total_empty_first_floor

# tools = [empty_parking_byfloor]
# llm_with_tools = llm.bind_tools(tools)
# tool_map = {tool.name: tool for tool in tools}

def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls


from typing import List, Literal

from langchain_community.chat_models import ChatOllama
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.tools import tool
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langgraph.graph import END, MessageGraph
from langgraph.prebuilt import ToolNode
    

def router(state: List[BaseMessage]) -> Literal["empty_parking_byfloor", "__end__"]:
    last_message = state[-1]
    tool_calls = last_message.tool_calls if isinstance(last_message, AIMessage) else []
    if len(tool_calls):
        return "empty_parking_byfloor"
    else:
        return "__end__"


# model = ChatOllama(model="llama3", temperature=0)
model = OllamaFunctions(model="llama3", temperature=0, format="json")
model_with_tools = model.bind_tools(
    tools=[
        {
            "name": "empty_parking_byfloor",
            "description": "return fetch empty Parking by floor number",
            "parameters": {
                "type": "object",
                "properties": {
                    "floor": {
                        "type": "int",
                        "description": "floor number, " "e.g. 1",
                    }
                },
                "required": ["floor"],
            },
        }
    ],
    function_call={"name": "empty_parking_byfloor"},
)

builder = MessageGraph()

builder.add_node("oracle", model_with_tools)

tool_node = ToolNode([empty_parking_byfloor])
builder.add_node("empty_parking_byfloor", tool_node)

builder.add_edge("empty_parking_byfloor", END)
builder.set_entry_point("oracle")

builder.add_conditional_edges("oracle", router)
runnable = builder.compile()

output = runnable.invoke("fetch empty Parking by floor number 1")
print(output)

# from langchain.tools.render import format_tool_to_openai_function

# functions = [
#     format_tool_to_openai_function(f) for f in [
#         empty_parking_byfloor
#     ]
# ]

# llm = llm.bind(functions = functions)

In [409]:

loader = CSVLoader(file_path=sorted_file_path)
data = loader.load()

In [410]:
data[0:10]

[Document(page_content='Floor: 1\nParking Number: 2\nParking Status: Filled\nParking Type: Special Need\nMetadata: Near to Door', metadata={'source': 'sorted_parking_spots.csv', 'row': 0}),
 Document(page_content='Floor: 1\nParking Number: 43\nParking Status: Empty\nParking Type: VIP\nMetadata: Left Corner', metadata={'source': 'sorted_parking_spots.csv', 'row': 1}),
 Document(page_content='Floor: 1\nParking Number: 44\nParking Status: Empty\nParking Type: Special Need\nMetadata: Near to Door', metadata={'source': 'sorted_parking_spots.csv', 'row': 2}),
 Document(page_content='Floor: 1\nParking Number: 46\nParking Status: Empty\nParking Type: Special Need\nMetadata: Near to Door', metadata={'source': 'sorted_parking_spots.csv', 'row': 3}),
 Document(page_content='Floor: 1\nParking Number: 47\nParking Status: Filled\nParking Type: VIP\nMetadata: Near to Door', metadata={'source': 'sorted_parking_spots.csv', 'row': 4}),
 Document(page_content='Floor: 1\nParking Number: 53\nParking Status

# Document Extract

In [411]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
t_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0
)

splits = t_splitter.split_documents(data)

# Document VectorStore

In [ ]:
# Method 1
from langchain.vectorstores import Chroma

presist_directory = "docs/chroma/"

vectordb = Chroma.from_documents(
    documents= splits, 
    embedding= embedding,
    persist_directory= presist_directory
)

In [ ]:
# Method 2
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding= embedding
).from_loaders([loader])

# Document Retreival

In [ ]:
question = "get me empty nomal parking near to lift"

In [ ]:
result = vectordb.similarity_search(question)
result

In [ ]:
from IPython.display import display, Markdown

indexResult = index.query(question, llm=llm)
display(Markdown(indexResult))

# Document QnA

In [ ]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

chat_history = []

memory = ConversationBufferMemory()
qa = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=index.vectorstore.as_retriever(), 
    return_source_documents=True,
    return_generated_question=False,
    verbose=True,
)

# qa = RetrievalQA.from_chain_type(
#     llm,
#     retriever = vectordb.as_retriever(),
#     chain_type = "stuff"
# )

In [ ]:
chain = qa | runnable
# chain.invoke({"question": question})

In [ ]:
question = "Looking for Empty parking"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

In [ ]:
question = "Looking for VIP Empty parking which is near to lift on floor 1"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

In [ ]:
question = "Looking for VIP Empty parking which is near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

In [ ]:
question = "Looking for Normal Empty parking which is near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

In [ ]:
question = "Looking for Empty parking which is near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

In [ ]:
question = "find parking spots floor 2 and 3 near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

In [ ]:
question = "How many empty parking available on Floor 1?"
result = qa({"question": question, "chat_history":chat_history})
# result
# chain.invoke({"role":"user","content": question, "chat_history":chat_history})
display(Markdown(result["answer"]))
# display(Markdown(result["result"]))

# Agent

In [ ]:
import re

def parking_byfloor(inputs: dict) -> dict:
    """Total Empty Parking Spots by Floor number"""
    print(inputs)
    # floor = inputs.get("floor", "")
    match = inputs #re.search(r'\d+', floor)
    if match:
        _floor = int(match)
    else:
        raise ValueError("No valid floor number found in the input")

    first_floor_empty = df_sorted[(df_sorted['Floor'] == _floor) & (df_sorted['Parking Status'] == 'Empty')]
    total_empty_first_floor = len(first_floor_empty)
    return {"answer":str(total_empty_first_floor)}

In [ ]:
from langchain.agents import initialize_agent, Tool

# Define the tool
parking_tool = Tool(
    name="ParkingByFloor",
    func=parking_byfloor,
    description="Total number of empty parking spots by floor number"
)

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = [parking_tool]
agent = initialize_agent(
    tools, 
    llm=llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    handle_parsing_errors=True
    )

In [ ]:
def is_parking_by_floor_relevant(query):
  # Define keywords/phrases related to parking by floor
  keywords = ["how many empty parking"]
  # Check for keywords in query or retrieved documents
  for keyword in keywords:
    if keyword in query.lower():
      print("Matched")
      return True
  return False


In [ ]:
def handle_query(query):
  # Analyze retrieved information or keywords in the query
  if is_parking_by_floor_relevant(query.get("question")):
    # Call ParkingByFloor tool
    return agent.run(query.get("question"))
  else:
    # Use LLM for general conversation or other tools if defined
    return qa(query)

In [ ]:
question = "How many empty parking available on floor 1?" 
result = handle_query({"question": question, "chat_history":chat_history})
print(result)

In [ ]:
question = "Looking for Empty parking which is near to lift"
result = handle_query({"question": question, "chat_history":chat_history})
print(result["answer"])